In [53]:
import numpy as np
import plotly.express as px
from sklearn.decomposition import PCA
import hdbscan
import pandas as pd

In [54]:
date = "02052022"

In [55]:
df = pd.read_csv("Tweets" + date + ".csv")
df = df.loc[df['Retweeting'] == "$"]
ce = np.load(date + "_corpus_embeddings.npy")
he = np.load(date + "_hashtag_embeddings.npy")
ce = ce[df['Hashtags'] != '[]']
he = he[df['Hashtags'] != '[]']
df = df.loc[df['Hashtags'] != '[]']
print(len(df))
df.index = np.arange(0, len(df))
pca = PCA(n_components = 100)
ce_pcad = pca.fit_transform(ce)
print(pca.explained_variance_ratio_)
print(np.sum(pca.explained_variance_ratio_))
pca = PCA(n_components = 40)
he_pcad = pca.fit_transform(he)
print(pca.explained_variance_ratio_)
print(np.sum(pca.explained_variance_ratio_))
embeddings = np.concatenate([ce_pcad, he_pcad], axis = 1)
#embeddings = ce_pcad
#cluster = hdbscan.HDBSCAN(min_cluster_size = 12, metric = 'euclidean', cluster_selection_method='eom').fit(embeddings)

2504
[0.11746588 0.07753745 0.06567082 0.06018987 0.04714743 0.04224522
 0.03319772 0.02928629 0.02678365 0.02518804 0.0223567  0.02002192
 0.01902154 0.01640734 0.014639   0.01453726 0.01317917 0.01261853
 0.0115835  0.01108979 0.01063509 0.01019487 0.0098358  0.00911172
 0.00870613 0.00830372 0.00776264 0.0069973  0.00664834 0.00645602
 0.00609701 0.00596179 0.00577315 0.00542573 0.00523902 0.00497274
 0.00485709 0.00467923 0.00442287 0.00429822 0.00399582 0.00388305
 0.00374099 0.00357532 0.0035578  0.00344443 0.0033413  0.00316093
 0.00307428 0.00303906 0.00285026 0.00277067 0.00270431 0.00260208
 0.00252568 0.00236006 0.00226772 0.00219271 0.00217047 0.00211034
 0.00201059 0.00194082 0.00188631 0.00184289 0.00179477 0.00175591
 0.0017307  0.00169461 0.00160777 0.00157497 0.00156791 0.00153241
 0.00149444 0.00143675 0.00137922 0.00137012 0.00130598 0.00128881
 0.00127984 0.0012629  0.00119334 0.00118314 0.00116113 0.00113325
 0.00108968 0.00107718 0.00106255 0.00101848 0.00100943 0

In [56]:
cluster = hdbscan.HDBSCAN(min_cluster_size = 8, metric = 'euclidean', cluster_selection_method='eom').fit(embeddings)
print(np.unique(cluster.labels_)[-1])

21


In [57]:
import re
#Find all words in the topic
unique_words = []
word_occurunces = []
num_topics = len(np.unique(cluster.labels_))

for i in range(len(df['Content'])):
    print(100*i/len(df['Content']), end = "\r")
    tweet = df['Content'][i]
    label = cluster.labels_[i] + 1
    for word in (re.split(" |,|\!|\.|\?|\"|\n",tweet)):
        if word != "" and word != "https://t" and word[:3] != "co/":
            if word in unique_words:
                ind = unique_words.index(word)
                word_occurunces[ind][label] = word_occurunces[ind][label] + 1
            else:
                unique_words.append(word)
                word_occurunces.append(np.zeros(num_topics))
                word_occurunces[-1][label] = 1
                
#relative topic frequency
totals = []
for i in range(len(word_occurunces)):
    tot = word_occurunces[i].sum()
    totals.append(tot)
    word_occurunces[i] = word_occurunces[i] / tot
totals = np.array(totals)
unique_words = np.array(unique_words)
word_occurunces = np.array(word_occurunces)

unique_words_f = unique_words[totals > 4]
word_occurunces_f = word_occurunces[totals > 4]
print(len(unique_words_f))

topic_predictors = []
for label in np.unique(cluster.labels_):
    topic_predictors.append(str(unique_words_f[np.argsort(word_occurunces_f[:,label+1])[-10:]]))
    
topic_predictors = np.array(topic_predictors)
topic_predictors = topic_predictors[cluster.labels_ + 1]
print(len(topic_predictors))

20766006389776358574
2504


In [58]:
pca = PCA(n_components = 3)
xyz = pca.fit_transform(embeddings)
cluster_label = cluster.labels_
content_with_br = []
for tweet in df.Content:
    i = 0
    num_chars = 0
    while (i < len(tweet)):
        if tweet[i] == " " and num_chars > 30:
            tweet = tweet[:i] + "<br>" + tweet[i:]
            i += 4
            num_chars = 0
        i += 1
        num_chars += 1
    content_with_br.append(tweet)
df["Content"] = content_with_br
df['x'] = xyz[:,0]
df['y'] = xyz[:,1]
df['z'] = xyz[:,2]
df['cluster'] = cluster_label
df['Topic Predictors'] = topic_predictors

fig = px.scatter_3d(df, x = "x", y="y", z="z", hover_name = "Name", color="cluster", hover_data = {"x": False, "y":False, "z":False, "Content":True, "Topic Predictors":True})
fig.update(layout_coloraxis_showscale=False)
fig.update_traces(marker=dict(size=2))
fig.update_layout(
    scene = dict(
        xaxis = dict(visible=False),
        yaxis = dict(visible=False),
        zaxis = dict(visible=False),
    )
)
fig.write_html("test_hashtags.html")

In [59]:
df = pd.read_csv("Tweets" + date + ".csv")
df = df.loc[df['Retweeting'] == "$"]
ce = np.load(date + "_corpus_embeddings.npy")
ce = ce[df['Hashtags'] == '[]']
df = df.loc[df['Hashtags'] == '[]']
df.index = np.arange(0, len(df))
pca = PCA(n_components = 200)
ce_pcad = pca.fit_transform(ce)
print(pca.explained_variance_ratio_)
print(np.sum(pca.explained_variance_ratio_))
embeddings = ce_pcad

[0.11831037 0.09863859 0.05693945 0.04527569 0.04097782 0.03503473
 0.0305344  0.02718597 0.02525349 0.02386177 0.02101102 0.02034496
 0.01651567 0.0153907  0.01522642 0.0142732  0.01289163 0.01262383
 0.01181986 0.011151   0.01033909 0.01030575 0.00942979 0.00919538
 0.00889236 0.00792298 0.00755935 0.0074417  0.00699518 0.00671996
 0.00637901 0.00634516 0.00612737 0.00594692 0.0057208  0.00560435
 0.00530655 0.00487622 0.00464188 0.00449929 0.00448869 0.00417572
 0.00405322 0.00380332 0.00370054 0.00359923 0.00344706 0.0033449
 0.00321796 0.0031066  0.00304844 0.0029569  0.00283566 0.0026811
 0.00262129 0.0025769  0.00247874 0.00236547 0.00229622 0.00219638
 0.00214022 0.00210088 0.0020434  0.00200328 0.00194513 0.00191969
 0.00185893 0.00184688 0.00179484 0.00173277 0.00167665 0.00162699
 0.00161391 0.00153827 0.00152395 0.00151405 0.00144942 0.00140733
 0.00137814 0.00134658 0.00131231 0.00130143 0.0012616  0.00124155
 0.00121353 0.00118411 0.00115546 0.00113113 0.00112786 0.001083

In [60]:
cluster = hdbscan.HDBSCAN(min_cluster_size = 8, metric = 'euclidean', cluster_selection_method='eom').fit(embeddings)
print(np.unique(cluster.labels_)[-1])

23


In [61]:
#Find all words in the topic
unique_words = []
word_occurunces = []
num_topics = len(np.unique(cluster.labels_))

for i in range(len(df['Content'])):
    print(100*i/len(df['Content']), end = "\r")
    tweet = df['Content'][i]
    label = cluster.labels_[i] + 1
    for word in (re.split(" |,|\!|\.|\?|\"|\n",tweet)):
        if word != "" and word != "https://t" and word[:3] != "co/":
            if word in unique_words:
                ind = unique_words.index(word)
                word_occurunces[ind][label] = word_occurunces[ind][label] + 1
            else:
                unique_words.append(word)
                word_occurunces.append(np.zeros(num_topics))
                word_occurunces[-1][label] = 1
                
#relative topic frequency
totals = []
for i in range(len(word_occurunces)):
    tot = word_occurunces[i].sum()
    totals.append(tot)
    word_occurunces[i] = word_occurunces[i] / tot
totals = np.array(totals)
unique_words = np.array(unique_words)
word_occurunces = np.array(word_occurunces)

unique_words_f = unique_words[totals > 4]
word_occurunces_f = word_occurunces[totals > 4]
print(len(unique_words_f))

topic_predictors = []
for label in np.unique(cluster.labels_):
    topic_predictors.append(str(unique_words_f[np.argsort(word_occurunces_f[:,label+1])[-10:]]))
    
topic_predictors = np.array(topic_predictors)
topic_predictors = topic_predictors[cluster.labels_ + 1]
print(len(topic_predictors))

44908502097064111646
6676


In [62]:
pca = PCA(n_components = 3)
xyz = pca.fit_transform(embeddings)
cluster_label = cluster.labels_
content_with_br = []
for tweet in df.Content:
    i = 0
    num_chars = 0
    while (i < len(tweet)):
        if tweet[i] == " " and num_chars > 30:
            tweet = tweet[:i] + "<br>" + tweet[i:]
            i += 4
            num_chars = 0
        i += 1
        num_chars += 1
    content_with_br.append(tweet)
df["Content"] = content_with_br
df['x'] = xyz[:,0]
df['y'] = xyz[:,1]
df['z'] = xyz[:,2]
df['cluster'] = cluster_label
df['Topic Predictors'] = topic_predictors

fig = px.scatter_3d(df, x = "x", y="y", z="z", hover_name = "Name", color="cluster", hover_data = {"x": False, "y":False, "z":False, "Content":True, "Topic Predictors":True})
fig.update(layout_coloraxis_showscale=False)
fig.update_traces(marker=dict(size=2))
fig.update_layout(
    scene = dict(
        xaxis = dict(visible=False),
        yaxis = dict(visible=False),
        zaxis = dict(visible=False),
    )
)
fig.write_html("test_no_hashtags.html")